In [1]:
import pandas as pd
from qdrant_client import QdrantClient, models
from FlagEmbedding import BGEM3FlagModel
import os
import ast
import re
import json
import tqdm

c:\Users\Admin\miniconda3\envs\crypto_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qdrant = QdrantClient("http://localhost", port=6333)

In [6]:
info = qdrant.get_collection(collection_name="law_corpus_bge")
print(f"Số lượng điểm đã lưu: {info.points_count}")

Số lượng điểm đã lưu: 954


In [ ]:
from qdrant_client.models import ScrollRequest

scroll_result = qdrant.scroll(
    collection_name="law_corpus_bge",
    limit=5,
    with_payload=True
)

for point in scroll_result[0]:
    print(f"ID: {point.id}")
    print(f"Text: {point.payload.get('text')}")
    print(f"Vector: {point.vector}...")  # In 5 phần tử đầu của vector
    print("-" * 50)

In [ ]:
def convert_to_list(s):
    s = s.strip('[]')  # Xóa dấu ngoặc vuông
    elements = s.split()  # Tách thành từng phần tử
    return [int(element) for element in elements]

def convert_str_to_list(input_str):    
    try:
        result = ast.literal_eval(input_str)
        return result
    except (ValueError, SyntaxError):
        print("Input is not a valid Python literal.")
        return None

def split_text_keeping_sentences(text, max_word_count):
    # Tách văn bản thành các câu
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    current_word_count = 0

    for sentence in sentences:
        # Đếm số từ trong câu
        word_count = len(sentence.split())
        
        # Nếu thêm câu vào chunk hiện tại sẽ vượt quá số lượng từ tối đa
        if current_word_count + word_count > max_word_count:
            # Thêm chunk hiện tại vào danh sách chunks
            chunks.append(current_chunk.strip())
            current_chunk = sentence  # Bắt đầu một chunk mới
            current_word_count = word_count  # Đặt lại số lượng từ
        else:
            current_chunk += " " + sentence.strip() if current_chunk else sentence.strip()
            current_word_count += word_count

    # Thêm chunk còn lại nếu có
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [2]:
class QdrantSearch_bge:
    def __init__(self, host: str, collection_name: str, model_name: str, use_fp16: bool = True):
        self.client = QdrantClient(host)
        self.collection_name = collection_name
        self.model = BGEM3FlagModel(model_name, use_fp16=use_fp16)
        
    def encode_query(self, query_text: str):
        """Encode the query text into dense and sparse vectors"""
        emb = self.model.encode(query_text, return_dense=True, return_sparse=True, return_colbert_vecs=False)
        emb_sparse = emb['lexical_weights']
        dense_vec = emb['dense_vecs']
        
        indices = list(emb_sparse.keys())
        values = list(emb_sparse.values())
        
        return dense_vec, indices, values

    def search(self, query_text: str, limit: int = 25):
        """Perform the search in Qdrant with the given query text and retrieve up to 50 results"""
        dense_vec, indices, values = self.encode_query(query_text)
        
        prefetch = [
            models.Prefetch(
                query=dense_vec,
                using="dense",
                limit=limit,
            ),
            models.Prefetch(
                query=models.SparseVector(
                    indices=indices,
                    values=values
                ),
                using="sparse",
                limit=limit,
            ),
        ]
        
        results = self.client.query_points(
            self.collection_name,
            prefetch=prefetch,
            query=models.FusionQuery(
                fusion=models.Fusion.RRF,
            ),
            with_payload=True,
            limit=limit,
        )
        
        return results

In [ ]:
class QuestionInference:
    def __init__(self, csv_path, save_pair_path, qdrant_search):
        self.csv_path = csv_path
        self.save_pair_path = save_pair_path
        self.qdrant_search = qdrant_search
    
    def load_questions(self):
        """Load questions and question_ids from CSV file"""
        self.questions = pd.read_csv(self.csv_path)
    
    def infer_and_save(self):
        """Infer each question and save results to a .txt file"""
        file_name = "data_round1"
        with open(os.path.join(self.save_pair_path, file_name + '.json'), 'w') as output_file:
            for row in tqdm.tqdm(self.questions.itertuples(index=False)):
                question = row.question
                list_id = convert_to_list(row.cid)
                list_context = convert_str_to_list(row.context)
                # create_data for bge
                save_dict = {}
                save_dict["query"] = question
                save_dict["pos"] = []
                save_dict["neg"] = []
                for context in list_context:
                    chunk_context = split_text_keeping_sentences(text=context, max_word_count=400)
                    save_dict["pos"] += chunk_context

                results = self.qdrant_search.search(query_text=question, limit=25)
                for result in results.points:
                    infor_id = int(result.payload["infor_id"])
                    if infor_id in list_id:
                        continue
                    else:
                        text = result.payload["text"]
                        save_dict["neg"].append(text)

                output_file.write(json.dumps(save_dict,ensure_ascii=False) + '\n')
                
                

In [ ]:
qdrant_search = QdrantSearch_bge(
    host="http://localhost:6333",
    collection_name="law_with_bge_round1",
    model_name="BAAI/bge-m3",
    use_fp16=True
)

In [9]:
questions = pd.read_csv(r"D:\Data\Legal-Retrieval\data\train.csv")
questions

,question,context,cid,qid
0,Người học ngành quản lý khai thác công trình t...,"['Khả năng học tập, nâng cao trình độ\n- Khối ...",[62492],161615
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,['Nội dung lồng ghép vấn đề bình đẳng giới tro...,[151154],80037
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"['""Điều 20. Ưu đãi về thời gian miễn thuế, giả...",[75071],124074
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"['Điều kiện được hưởng\nCán bộ quản lý, giáo v...",[225897],146841
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"['Nguyên tắc áp dụng\n1. Trường hợp công chức,...",[68365],6176
...,...,...,...,...
119451,Thư ký Tòa án làm việc 5 năm có được bổ nhiệm ...,"['""Điều 93. Thẩm tra viên\n1. Thẩm tra viên là...",[51236],47570
119452,Nội dung đơn thuốc có quy định hay tùy vào tìn...,"['""1. Ghi đủ, rõ ràng và chính xác các mục in ...",[47787],8661
119453,Việc khai trương hoạt động đơn vị sự nghiệp củ...,['Khai trương hoạt động\n....\n2. Tổ chức tài ...,[176971],103122
119454,Chủ đầu tư không công khai báo cáo đánh giá tá...,['Vi phạm các quy định về đánh giá tác động mô...,[182633],157644


In [ ]:
questions